In [1]:
%load_ext lab_black

Step: Load data

In [1]:
import pandas as pd
import os
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit
import numpy as np
import pandas as pd
import gower
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
)


path_data = here("./data")
os.chdir(path_data)


data_draft_kings = pd.read_csv("draft_kings_2021_2022.csv")

In [2]:
data_draft_kings

,Position,Name,Points,Salary,Reduction,Value,Ownership %,Place,week
0,QB,Patrick Mahomes,36.28,8100,8.1,4.48,8.20%,1,1
1,RB,Melvin Gordon,23.80,5300,5.3,4.49,2.00%,1,1
2,RB,Joe Mixon,28.00,6200,6.2,4.52,16.00%,1,1
3,WR,Tyreke Hill,40.10,8200,8.2,4.89,10.80%,1,1
4,WR,Corey Davis,26.70,4900,4.9,5.45,12.20%,1,1
...,...,...,...,...,...,...,...,...,...
562,WR,Mike Evans,29.70,7000,7.0,4.24,32.5%,3,20
563,WR,Tyreke Hill,16.70,7300,7.3,2.29,33.3%,3,20
564,TE,Travis Kelce,28.88,6700,6.7,4.31,29.1%,3,20
565,FLEX,Demarcus Robinson,11.60,3300,3.3,3.52,1.2%,3,20


In [28]:
import pulp
import pandas as pd

def optimize_draft_kings_lineup(df, num_players):
    # Create the problem variable
    prob = pulp.LpProblem("DraftKings_Milly_Makers", pulp.LpMaximize)

    # Create decision variables
    player_vars = pulp.LpVariable.dicts("Player", (player for player in df["name"]), 0, 1, pulp.LpInteger)

    # Objective function: maximize total value
    prob += pulp.lpSum([df.loc[df["name"] == player, "value"].values[0] * player_vars[player] for player in df["name"]]), "Total_Value"

    # Constraints
    prob += pulp.lpSum([df.loc[df["name"] == player, "cost"].values[0] * player_vars[player] for player in df["name"]]) <= 50000, "Total_Cost"
    
    for position, count in num_players.items():
        prob += pulp.lpSum([player_vars[player] for player in df[df["position"] == position]["name"]]) == count, f"{count}_{position}s"

    # Solve the problem
    prob.solve()

    # Store the results
    optimal_lineup = [v.name.split("_")[1] for v in prob.variables() if v.varValue > 0]

    # Return the optimal lineup as a DataFrame
    return df[df["name"].isin(optimal_lineup)]

# Sample data as a DataFrame
data = {
    "name": ["Player1", "Player2", "Player3", "Player4", "Player5", "Player6", "Player7", "Player8"],
    "position": ["QB", "QB", "RB", "RB", "WR", "WR", "TE", "TE"],
    "cost": [6000, 6100, 5000, 5200, 4500, 4800, 4300, 4600],
    "value": [20, 22, 15, 18, 12, 14, 10, 13]
}

df = pd.DataFrame(data)

# Set the number of players for each position
num_players = {"QB": 1, "RB": 2, "WR": 3, "TE": 1}

# Run the optimization and print results
optimal_lineup_df = optimize_draft_kings_lineup(df, num_players)
print("Optimal Lineup:")
print(optimal_lineup_df)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /anaconda/envs/azureml_py38/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5f8a0e3beade44b994cfca4310863bc8-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/5f8a0e3beade44b994cfca4310863bc8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 51 RHS
At line 57 BOUNDS
At line 66 ENDATA
Problem MODEL has 5 rows, 8 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Optimal Lineup:
      name position  cost  value
1  Player2       QB  6100     22
2  Player3       RB  5000     15
3  Player4       RB  5200     18
4  Player5       WR  4500     12
5  Player6       WR  4800     14
7  Player8       TE  46

In [29]:
df.to_csv("data_draft_kings_optimize_test.csv")